In [0]:
# Importing all the required packages.

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


In [0]:
with open('train + dev.txt', 'r') as f:
  data = f.read()

In [0]:
with open('test.txt', 'r') as f:
  test_data = f.read()

In [0]:
labels,texts = [],[]
for i,line in enumerate(data.split("\n")):
  content = line.split()
  labels.append(content[-1])
  texts.append(" ".join(content[1:-1]))

In [0]:
test_labels,test_texts = [],[]
for j,row in enumerate(test_data.split("\n")):
  content = row.split()
  test_labels.append(content[-1])
  test_texts.append(" ".join(content[1:-1]))

In [0]:
print(test_texts[0])

Hmm What does your bio mean? I don’t have any bio


In [0]:
# create a dataframe using texts and lables
trainDF = pandas.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

In [0]:
# create a dataframe using texts and lables for test data
testDF = pandas.DataFrame()
testDF['test_text'] = test_texts
testDF['test_label'] = test_labels

In [0]:
print(type(trainDF['text']))

<class 'pandas.core.series.Series'>


In [0]:
print(labels[26])

sad


In [0]:
print(labels[1])

angry


In [0]:
print(texts[2])

By by Google Chrome Where you live


In [0]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)
test_y = encoder.fit_transform(testDF['test_label'])


In [0]:
print(type(train_y))
print(type(test_y))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [0]:
print(type(train_x))

<class 'pandas.core.series.Series'>


In [0]:
print(test_y[0])

2


In [0]:
print(train_y)

[0 2 3 ... 2 3 0]


In [0]:
print(len(train_x))
print(len(valid_x))

24686
8229


In [0]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [0]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [0]:
# train a LDA Model
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

In [0]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [0]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print ("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("NB, CharLevel Vectors: ", accuracy)

NB, Count Vectors:  0.7375136711629603
NB, WordLevel TF-IDF:  0.6756592538583059
NB, N-Gram Vectors:  0.709928302345364
NB, CharLevel Vectors:  0.707740916271722


In [0]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(xtrain_tfidf_ngram, train_y)

In [0]:
# SVM implementation on Ngram
accuracy = train_model(svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto'), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)

#SVM implementation on Count vectors
accuracy = train_model(svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto'), xtrain_count, train_y, xvalid_count)
print("SVM, Count Vectors: ", accuracy)

#SVM on character level tf idfvectors
accuracy = train_model(svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto'), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("SVM, CharLevel Vectors: ", accuracy)

SVM, N-Gram Vectors:  0.732895856118605
SVM, Count Vectors:  0.7842994288491919
SVM, CharLevel Vectors:  0.8121278405638596


# **TEst DATA**


In [0]:
# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 